In [57]:
import os
import httpx
from dotenv import load_dotenv
import psycopg2
import json

1. Open the link --> https://openweathermap.org/api
2. Click on the *API doc* for Current Weather Data - https://openweathermap.org/current
3. load_dotenv() - Load environment variables from .env file

In [58]:
def load_env_variables():
    load_dotenv()
    api_key_c = os.getenv("api_key_c")
    api_key_w = os.getenv("api_key_w")
    db_name = os.getenv("DB_NAME")
    db_user = os.getenv("DB_USER")
    db_password = os.getenv("DB_PASSWORD")
    db_host = os.getenv("DB_HOST")
    db_port = os.getenv("DB_PORT")
    return api_key_c, api_key_w, db_name, db_user, db_password, db_host, db_port

In [65]:
print(load_env_variables())

('ekUxalRhaW1PNUw3YlZ1bjRVYzRFTzl3RktNMkdSYXBnNExrRnN6Qw==', 'cc2f7b4593e7891a183e06c68fbe1535', 'weather', 'postgres', '19SIMba97.', 'localhost', '5432')


In [59]:
def fetch_cities(api_key):
    base_url_c = "https://api.countrystatecity.in/v1/countries/MY/cities"
    headers = {"X-CSCAPI-KEY": api_key}
    c_response = httpx.get(base_url_c, headers=headers)
    c_response.raise_for_status()
    return [city['name'] for city in c_response.json()]

In [67]:
print(fetch_cities(api_key_c))

['Alor Gajah', 'Alor Setar', 'Ampang', 'Ayer Hangat', 'Bagan Pulau Ketam', 'Bagan Serai', 'Bahau', 'Bakri', 'Bandar Labuan', 'Banting', 'Batang Berjuntai', 'Batu Arang', 'Batu Berendam', 'Batu Feringgi', 'Batu Gajah', 'Batu Pahat', 'Beaufort', 'Bedong', 'Bemban', 'Bentong Town', 'Bidur', 'Bintulu', 'Bukit Mertajam', 'Bukit Rambai', 'Buloh Kasap', 'Butterworth', 'Chaah', 'Cukai', 'Daerah Batu Pahat', 'Daerah Johor Baharu', 'Daerah Keluang', 'Daerah Kota Tinggi', 'Daerah Mersing', 'Daerah Muar', 'Daerah Pontian', 'Daerah Segamat', 'Data Kakus', 'Donggongon', 'George Town', 'Gua Musang', 'Gurun', 'Ipoh', 'Jenjarum', 'Jerantut', 'Jertih', 'Jitra', 'Johor Bahru', 'Juru', 'Kampar', 'Kampong Baharu Balakong', 'Kampong Dungun', 'Kampong Kadok', 'Kampong Masjid Tanah', 'Kampong Pangkal Kalong', 'Kampung Ayer Keroh', 'Kampung Ayer Molek', 'Kampung Baharu Nilai', 'Kampung Baru Subang', 'Kampung Batu Feringgi', 'Kampung Bukit Baharu', 'Kampung Bukit Tinggi Bentong', 'Kampung Kilim', 'Kampung Kok',

In [60]:
def fetch_weather(api_key, city):
    base_url_w = "https://api.openweathermap.org/data/2.5/weather"
    params = {'appid': api_key, 'q': city, 'units': "metric"}
    w_response = httpx.get(base_url_w, params=params)
    w_response.raise_for_status()
    return w_response.json()

In [68]:
fetch_weather(api_key_w, "Ampang")

{'coord': {'lon': 101.768, 'lat': 3.1498},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 27.45,
  'feels_like': 32.19,
  'temp_min': 26.44,
  'temp_max': 27.79,
  'pressure': 1008,
  'humidity': 91,
  'sea_level': 1008,
  'grnd_level': 1002},
 'visibility': 10000,
 'wind': {'speed': 0.84, 'deg': 74, 'gust': 1.06},
 'clouds': {'all': 100},
 'dt': 1720895035,
 'sys': {'type': 2,
  'id': 66184,
  'country': 'MY',
  'sunrise': 1720912200,
  'sunset': 1720956430},
 'timezone': 28800,
 'id': 1733046,
 'name': 'Ampang',
 'cod': 200}

In [61]:
def get_db_connection_and_cursor(db_name, db_user, db_password, db_host, db_port):
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    cursor = conn.cursor()
    return conn, cursor

In [62]:
def create_weather_table(cursor):
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS weather_jsondata (
            id SERIAL PRIMARY KEY,
            weather_data JSONB NOT NULL
        )
    """)

In [63]:
def insert_weather_data(cursor, weather_data):
    cursor.execute("INSERT INTO weather_jsondata (weather_data) VALUES (%s)", (json.dumps(weather_data),))

In [64]:
def main():
    api_key_c, api_key_w, *db_params = load_env_variables()
    conn, cursor = get_db_connection_and_cursor(*db_params)
    create_weather_table(cursor)
    conn.commit()  # Commit the transaction after table creation

    cities = fetch_cities(api_key_c)
    for city in cities:
        try:
            weather_data = fetch_weather(api_key_w, city)
            insert_weather_data(cursor, weather_data)
            conn.commit()
        except (httpx.RequestError, httpx.HTTPStatusError, ValueError):
            continue

    cursor.close()
    conn.close()


main()

In [ ]:
import os
import httpx
from dotenv import load_dotenv
import psycopg2
import json

class WeatherDataProcessor:
    def __init__(self):
        self.api_key_c, self.api_key_w, *db_params = self.load_env_variables()
        self.conn, self.cursor = self.get_db_connection_and_cursor(*db_params)

    def load_env_variables(self):
        load_dotenv()
        api_key_c = os.getenv("api_key_c")
        api_key_w = os.getenv("api_key_w")
        db_name = os.getenv("DB_NAME")
        db_user = os.getenv("DB_USER")
        db_password = os.getenv("DB_PASSWORD")
        db_host = os.getenv("DB_HOST")
        db_port = os.getenv("DB_PORT")
        return api_key_c, api_key_w, db_name, db_user, db_password, db_host, db_port

    def fetch_cities(self):
        base_url_c = "https://api.countrystatecity.in/v1/countries/MY/cities"
        headers = {"X-CSCAPI-KEY": self.api_key_c}
        c_response = httpx.get(base_url_c, headers=headers)
        c_response.raise_for_status()
        return [city['name'] for city in c_response.json()]

    def fetch_weather(self, city):
        base_url_w = "https://api.openweathermap.org/data/2.5/weather"
        params = {'appid': self.api_key_w, 'q': city, 'units': "metric"}
        w_response = httpx.get(base_url_w, params=params)
        w_response.raise_for_status()
        return w_response.json()

    def get_db_connection_and_cursor(self, db_name, db_user, db_password, db_host, db_port):
        conn = psycopg2.connect(
            dbname=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port
        )
        cursor = conn.cursor()
        return conn, cursor

    def create_weather_table(self):
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS weather_jsondata (
                id SERIAL PRIMARY KEY,
                weather_data JSONB NOT NULL
            )
        """)

    def insert_weather_data(self, weather_data):
        self.cursor.execute("INSERT INTO weather_jsondata (weather_data) VALUES (%s)", (json.dumps(weather_data),))

    def process_weather_data(self):
        self.create_weather_table()  # Create the table

        self.conn.commit()  # Commit the transaction after table creation

        cities = self.fetch_cities()
        for city in cities:
            try:
                weather_data = self.fetch_weather(city)
                self.insert_weather_data(weather_data)
                self.conn.commit()  # Commit after each insert
            except (httpx.RequestError, httpx.HTTPStatusError, ValueError):
                continue  # Skip printing errors

    def close_connection(self):
        self.cursor.close()
        self.conn.close()

WeatherDataProcessor()
    


the code below is created for logic

In [ ]:
load_dotenv()

In [ ]:
api_key_c = os.getenv("api_key_c")
api_key_w = os.getenv("api_key_w")

In [ ]:
base_url_c = "https://api.countrystatecity.in/v1/countries/MY/cities"
headers = {"X-CSCAPI-KEY": api_key_c}
response = httpx.get(base_url_c, headers=headers).json()
cities = [city['name'] for city in response]
print(cities)
num_cities = len(cities)
print(num_cities)


In [ ]:
weather_data = []
w_url = "https://api.openweathermap.org/data/2.5/weather"
for city in cities:
    params = {'appid': api_key_w, 'q': city, 'units': "metric"}
    try:  
        w_response = httpx.get(w_url, params=params)
        w_response.raise_for_status()
        city_weather = {
            'city': city,
            'wd': w_response.json()
        }
        weather_data.append(city_weather)
        # print(f"Weather data fetched for {city}")
    except (httpx.RequestError, httpx.HTTPStatusError, ValueError):
        pass

In [ ]:
output_file = 'weather_data.json'
with open(output_file, 'w') as f:
    json.dump(weather_data, f, indent=2)

In [ ]:
conn = psycopg2.connect(
            dbname=os.getenv("DB_NAME"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            host=os.getenv("DB_HOST"),
            port=os.getenv("DB_PORT")
        )
cursor = conn.cursor()

In [ ]:
cursor.execute("""
        CREATE TABLE IF NOT EXISTS weather_data (
            id SERIAL PRIMARY KEY,
            weather_data JSONB NOT NULL
        );
    """)

In [ ]:
# Insert data into the table
for data in weather_data:
    weather_json = json.dumps(data['wd'])
    cursor.execute("INSERT INTO weather_data (weather_data) VALUES (%s)", (weather_json,))


In [ ]:
cursor.execute("CREATE TABLE stag_weather AS (SELECT * FROM weather_data)")

In [ ]:
conn.commit()
conn.close()

In [ ]:
import psycopg2
from psycopg2 import sql
import os

# Connection details from environment variables
conn_params = {
    'dbname': os.getenv("DB_NAME"),
    'user': os.getenv("DB_USER"),
    'password': os.getenv("DB_PASSWORD"),
    'host': os.getenv("DB_HOST"),
    'port': os.getenv("DB_PORT")
}

try:
    # Connect to the database
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()

    # Drop the table if it exists
    drop_table_query = sql.SQL("DROP TABLE IF EXISTS {table}").format(
        table=sql.Identifier('stag_weather')
    )
    cursor.execute(drop_table_query)
    conn.commit()

    print("Table 'weather_data' deleted successfully.")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


In [56]:
import os
import httpx
from dotenv import load_dotenv
import psycopg2
import json

def load_env_variables():
    load_dotenv()
    api_key_c = os.getenv("api_key_c")
    api_key_w = os.getenv("api_key_w")
    db_name = os.getenv("DB_NAME")
    db_user = os.getenv("DB_USER")
    db_password = os.getenv("DB_PASSWORD")
    db_host = os.getenv("DB_HOST")
    db_port = os.getenv("DB_PORT")
    return api_key_c, api_key_w, db_name, db_user, db_password, db_host, db_port

def fetch_cities(api_key):
    base_url_c = "https://api.countrystatecity.in/v1/countries/MY/cities"
    headers = {"X-CSCAPI-KEY": api_key}
    c_response = httpx.get(base_url_c, headers=headers)
    c_response.raise_for_status()
    return [city['name'] for city in c_response.json()]

def fetch_weather(api_key, city):
    base_url_w = "https://api.openweathermap.org/data/2.5/weather"
    params = {'appid': api_key, 'q': city, 'units': "metric"}
    w_response = httpx.get(base_url_w, params=params)
    w_response.raise_for_status()
    return w_response.json()

def get_db_connection_and_cursor(db_name, db_user, db_password, db_host, db_port):
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    cursor = conn.cursor()
    return conn, cursor

def create_weather_table(cursor):
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS weather_jsondata (
            id SERIAL PRIMARY KEY,
            weather_data JSONB NOT NULL
        )
    """)

def insert_weather_data(cursor, weather_data):
    cursor.execute("INSERT INTO weather_jsondata (weather_data) VALUES (%s)", (json.dumps(weather_data),))

def main():
    api_key_c, api_key_w, *db_params = load_env_variables()

    conn, cursor = get_db_connection_and_cursor(*db_params)

    create_weather_table(cursor)  # Create the table

    conn.commit()  # Commit the transaction after table creation

    cities = fetch_cities(api_key_c)
    for city in cities:
        try:
            weather_data = fetch_weather(api_key_w, city)
            insert_weather_data(cursor, weather_data)
            conn.commit()  # Commit after each insert
        except (httpx.RequestError, httpx.HTTPStatusError, ValueError):
            continue

    cursor.close()
    conn.close()


main()